In [65]:
import pandas
import numpy as np

def encode_seq(sequence):
	alphabet = ['A', 'C', 'D', 'E', 'F', 'G','H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
	char_to_int = dict((c, i) for i, c in enumerate(alphabet))
	integer_encoded = np.array([char_to_int[char] for char in sequence])
	return integer_encoded

with open('temp.csv', 'r') as file:
	data = pandas.read_csv(file, nrows=100000)
	data.drop(columns=['Target', 'Assay', 'Replicate'], inplace=True)

data['Sequence'] = data['Sequence'].apply(encode_seq)
data['HC'] = data['HC'].apply(encode_seq)
data['LC'] = data['LC'].apply(encode_seq)
data['CDRH1'] = data['CDRH1'].apply(encode_seq)
data['CDRH2'] = data['CDRH2'].apply(encode_seq)
data['CDRH3'] = data['CDRH3'].apply(encode_seq)
data['CDRL1'] = data['CDRL1'].apply(encode_seq)
data['CDRL2'] = data['CDRL2'].apply(encode_seq)
data['CDRL3'] = data['CDRL3'].apply(encode_seq)

In [70]:
import sklearn.ensemble

train_data, test_data = sklearn.model_selection.train_test_split(data, test_size=0.2)

X_train = np.array(train_data['Sequence'].tolist())
y_train = np.array(train_data['Pred_affinity'].tolist())
X_test = np.array(test_data['Sequence'].tolist())
y_test = np.array(test_data['Pred_affinity'].tolist())


param_grid = {
	'n_estimators': [200, 300, 400],
	'max_depth': [40, 50, 60],
	'min_samples_split': [2],
	'min_samples_leaf': [1],
}

model = sklearn.ensemble.RandomForestRegressor()
grid_search = sklearn.model_selection.GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=3)
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_score_)
model = grid_search.best_estimator_
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(sklearn.metrics.mean_squared_error(y_test, y_pred))
print(sklearn.metrics.r2_score(y_test, y_pred))
print(y_pred[:10])
print(y_test[:10])

Fitting 3 folds for each of 12 candidates, totalling 36 fits
